<a href="https://colab.research.google.com/github/unclepete-20/lab-06-07-md/blob/main/Red_Neuronal_Basica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio 6 y 7 (Redes Neuronales)
## Regresión Lineal Simple. Ejemplo minimalista

Integrantes:
- Pedro Pablo Arriola Jimenez (20188)
- Jose Rodrigo Barrera Garcia (20807)

### Importar las librerías relevantes

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # Para graficar en 3-D
#Importamos time
import time

### Generar datos al azar para entrenar al modelo

Trabajaremos con dos variables de entrada, las x1 y x2 en nuestros ejemplos anteriores. Se generan al azar a partir de una distribución uniforme.

Se creará una matriz con estas dos variables.  La matriz X del modelo lineal y = x * w + b

In [2]:
# Por facilidad, declaramos una variable que indique el tamaño del conjunto 
#      de datos de entrenamiento.
#observaciones = 1000
#Hacemos el cambio a 100,000 observaciones
observaciones = 1000

x1 = np.random.uniform(low=-10, high=10, size=(observaciones,1))
x2 = np.random.uniform(-10, 10, (observaciones,1))

X = np.column_stack((x1,x2))

# Verificar la forma de la matriz 
# Debiera ser n x k, donde n es el número de observaciones, y k es el número de variables.
print (X.shape)

(1000, 2)


### Generar las metas a las que debemos apuntar

Inventaremos una función f(x1, x2) = 2 * x1 - 3 * x2 + 5 + <ruido pequeño>.  El ruido es para hacerlo más realista.

Utilizaremos la metodología de ML, y veremos si el algoritmo la ha aprendido.  

In [3]:
ruido = np.random.uniform(-1, 1, (observaciones,1))

print(ruido)

#metas = 2 * x1 - 3 * x2 + 5 + ruido
#Nueva función metas
metas = 13 * x1 + 7 * x2 - 12

# Veamos las dimensiones. Deben ser n x m, donde m es el número de variables de salida.
print (metas.shape)

[[ 5.46443104e-01]
 [ 4.38482007e-01]
 [-7.67022689e-01]
 [-9.70487294e-02]
 [ 5.01062402e-01]
 [-8.08505652e-01]
 [-1.51157272e-01]
 [ 8.12798720e-01]
 [-2.01756503e-01]
 [-1.99353078e-01]
 [-9.64460579e-01]
 [ 7.29071881e-01]
 [-3.16072575e-01]
 [ 9.69081768e-02]
 [-8.70874419e-01]
 [-1.91197086e-01]
 [-3.52573478e-01]
 [ 8.08411473e-01]
 [ 9.37410288e-01]
 [-3.23773971e-01]
 [-8.84171261e-01]
 [ 4.79941013e-01]
 [ 5.53021150e-01]
 [ 5.34550122e-01]
 [ 4.65868906e-01]
 [-5.25760535e-01]
 [ 5.04949063e-01]
 [-2.44255441e-01]
 [-7.08935308e-01]
 [ 5.31944989e-02]
 [ 9.61336140e-01]
 [-5.19327417e-01]
 [-9.63205062e-01]
 [ 2.43702516e-01]
 [ 1.24774271e-01]
 [ 5.18961721e-01]
 [ 9.08313307e-01]
 [ 3.57531619e-01]
 [ 1.26395812e-01]
 [-1.53054650e-01]
 [-7.06121551e-01]
 [-7.55452722e-01]
 [ 1.27674164e-01]
 [-6.38660362e-02]
 [ 8.87795997e-02]
 [-8.58635059e-01]
 [ 6.02764468e-01]
 [ 8.59807869e-01]
 [-2.73764443e-01]
 [ 1.90187129e-01]
 [-2.33516854e-01]
 [-1.83492357e-01]
 [-4.7489661

### Graficar los datos a usar para el entrenamiento

La idea es ver que haya una fuerte tendencia que nuestro modelo debe aprender a reproducir.


In [4]:
print(x1.shape)
print(x2.shape)
print(metas.shape)

(1000, 1)
(1000, 1)
(1000, 1)


In [5]:
x1N = x1.reshape(observaciones,)
x2N = x2.reshape(observaciones,)
metasN = metas.reshape(observaciones,)

fig = px.scatter_3d(x = x1N, y = x2N, z = metasN)

fig.update_layout(
    width = 500,
    height = 500,)

fig.show()

### Inicializar variables

Inicializaremos los pesos y sesgos, al azar, dentro de un rango inicial pequeño.  Es posible "jugar" con este valor pero no es recomendable ya que el uso de rangos iniciales altos inhibe el aprendizaje por parte del algoritmo

Los pesos son de dimensiones k x m, donde k es el numero de variables de entrada y m es el número de variables de salida.  

Como solo hay una salida, el sesgo es de tamaño 1, y es un escalar

In [6]:
rango_inicial = 0.1

pesos = np.random.uniform(low = -rango_inicial, high = rango_inicial, size=(2, 1))

sesgos = np.random.uniform(low = -rango_inicial, high = rango_inicial, size=1)

#Veamos cómo fueron inicializados.
print (pesos)
#print (sesgos)

[[-0.08708275]
 [ 0.04058099]]


In [7]:
pesos.shape

(2, 1)

### Asignar la tasa de aprendizaje (Eta)

Se asigna un a tasa de aprendizaje pequeña.  Para este ejemplo funciona bien 0.02.  Vale la pena "jugar" con este valor para ver los resultados de hacerlo.

In [8]:
#Tasa de aprendizaje
eta = 0.0001

### Entrenar el modelo

Usaremos un valor de 100 para iterar sobre el conjunto de datos de entrenamiento.  Ese valor funciona bastante bien con la tasa de aprendizaje de 0.02.  Cómo saber el número adecuado de iteraciones es algo que veremos en futuras sesiones, pero generalmente una tasa de aprendizaje baja requiere de más iteraciones que una más alta.  Sin embargo hay que tener en mente que una tasa de aprendizaje alta puede causar que la pérdida "Loss" diverja a infinito, en vez de converger a cero (0)

Usaremos la función de pérdida L2-norm, pero dividido por 2, para ser consistente con la clase.  Es más, también lo dividiremos por el número de observaciones para obtener un promedio de pérdida por observación.  Hablamos en clase sobre la posiilidad de modificar esta función una vez no se pierda la característica de ser más baja para los resultados mejores, y vice versa.

Imprimimos la función de pérdida (loss) en cada iteración, para ver si está decreciendo como se desea.

Otro pequeño truco es escalar las deltas de la misma manera que se hizo con la función de pérdida.  De esta forma la tasa de aprendizaje es independiente del número de muestras (samples u observaciones).  De nuevo esto no cambia el principio, solo hace más fácil la selección de una tasa única de aprendizaje. 

Finalmente aplicamos la regla de actualización del decenso de gradiente.

Ojo!  los pesos son 2x1, la tasa de aprendizaje es 1x1 (escalar), las entradas son 1000x2, y las deltas escaladas son 1000x1.  Necesitamos obtener la transpuesta de las entradas para que no hayan problemas de dimensión en las operaciones. 



In [9]:
#Inicializamos el tiempo inicial
tiempo_inicial = time.time()
for i in range (100):
    
    # Esta es la ecuacion del modelo lineal: y = xw + b 
    y = np.dot(X, pesos) + sesgos
    
    # Las deltas son las diferencias entre las salidas y las metas (targets)
    # deltas es un vector 1000 x 1
    deltas = y - metas
        
    #perdida = np.sum(deltas ** 2) / 2 / observaciones
    #Perdida sin estar dividido por 2
    #perdida = np.sum(deltas ** 2) / observaciones
    #Lo cambiamos por la función pérdida L1-norm
    perdida = np.sum(np.abs(deltas)) / observaciones
    
    print("Perdida: ",perdida)
    
    deltas_escaladas = deltas / observaciones
      
    pesos = pesos - eta * np.dot(X.T, deltas_escaladas)
    sesgos = sesgos - eta * np.sum(deltas_escaladas)
    
    # Los pesos son actualizados en una forma de algebra lineal(una matriz menos otra)
    # Sin embargo, los sesgos en este caso son solo un número (solo estamos calculando una salida), 
    #       es necesario transformar las deltas a un escalar.      
    # Ambas lineas son consistentes con la metodología de decenso de gradiente-
#Declaramos el tiempo final de ejecución
tiempo_final = time.time()
#Calculamos el tiempo de ejecución
tiempo_ejecucion = tiempo_final - tiempo_inicial
print('El tiempo de ejecución fue:',tiempo_ejecucion)


Perdida:  233.21894430585456
Perdida:  231.7531743942974
Perdida:  230.29713650464396
Perdida:  228.85076583664662
Perdida:  227.41399802210245
Perdida:  225.98676912196842
Perdida:  224.5690156234964
Perdida:  223.1606744373871
Perdida:  221.76168289496277
Perdida:  220.37197874535948
Perdida:  218.99150015273742
Perdida:  217.62018569351028
Perdida:  216.25797435359317
Perdida:  214.90480552566868
Perdida:  213.56061900647146
Perdida:  212.22535499409076
Perdida:  210.8989540852914
Perdida:  209.5813572728519
Perdida:  208.27250594292116
Perdida:  206.9723418723925
Perdida:  205.6808072262952
Perdida:  204.39784455520356
Perdida:  203.12339679266327
Perdida:  201.8574072526349
Perdida:  200.59981962695474
Perdida:  199.35057798281224
Perdida:  198.10962676024457
Perdida:  196.87691076964805
Perdida:  195.65237518930564
Perdida:  194.43596556293167
Perdida:  193.22762779723254
Perdida:  192.02730815948382
Perdida:  190.8349532751236
Perdida:  189.65051012536202
Perdida:  188.473926044

### Desplegamos los pesos y el sesgo para ver si funcionaron correctamente.

Por el diseño de nuestro datos, los pesos debieran ser 2 y -3, y el sesgo: 5

**NOTA:**  Si aún no están los valores correctos, puede que aún estén convergiendo y sea necesario iterar más veces.  Para esto solo se requiere ejecutar la celda anterior cuantas veces sea requerido

In [10]:
print(pesos, sesgos)      

[[ 0.47297584]
 [-0.82568485]] [0.00225333]


### Graficar las últimas salidas vrs las metas (targets)

Como son las últimas, luego del entrenamiento, representan el modelo final de exactitud.  Entre más cercana esté esta gráfica a una línea de 45 grados, más cercanas están las salidas y metas.

Como este ejemplo es pequeño, es posible hacerlo, en los problemas que veremos más tarde en la clase, esto ya no sería posible.

In [11]:
yN = y.reshape(observaciones,)
metasN = metas.reshape(observaciones,)
fig = px.scatter(x = yN, y =  metasN)

fig.update_layout(
    width = 400,
    height = 400,)

fig.show()